# Scrapeando tango (1 de varios)

## Bajando información de los "creadores"

La idea de este post es mostrar una forma bien sencilla y rápida de webscraping. Particularmente, estamos trabajando en un proyecto para hacer análisis de letras de tango con NLP. En ese sentido, los datos relativos a los letristas, compositores, cantores y demás figuras dentro del tango probablemente resulten relevantes.

Este notebook muestra una implementación de una función simple de scraping.

## La fuente: [Todo Tango](http://www.todotango.com/)

El sitio [Todo Tango](http://www.todotango.com/) tiene una gran cantidad de información acerca de letras, compositores, músicos, cantores y letristas de tango. Pero, además, tiene una estructura que la hace bastante fácil de scrapear (shhhhh...). 

Básicamente, podemos acceder al link "maestro" que contiene todos los links de los llamados "creadores" (es decir, todas las figuras que nos interesan). 

* http://www.todotango.com/creadores/

Ahí podemos ver que hay varias solapas por tipo de "creador". Pero existe una que es "Todos". 

Luego, se nota que hay una solapa por cada letra del apellido. Pero, una vez más, hay una solapa ["\*"] que contiene todos los links de los creadores.

* http://www.todotango.com/creadores/listar/all/all/

Entonces, lo que vamos a tener que hacer es lo siguiente:

1. scrapear todos los links que van a cada una de las páginas de los creadores
2. entrar a todas las páginas de los creadores y scrapear la información que nos interesa

Lo bueno es que la información adentro de cada una de las páginas está bastante estructurada y es bastante fácil de scrapear.

## Cómo hacerlo...

**1) Primero, importamos los paquetes a utilizar**

In [1]:
import urllib2
from bs4 import BeautifulSoup as soup
from numpy import random
import multiprocessing as mp
import json, codecs

**2) Usando [`urlib2`](https://docs.python.org/2/library/urllib2.html) y [`BeautifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#) generamos una lista que contiene todos los links en la página de "los creadores"**

In [2]:
index = urllib2.urlopen('http://www.todotango.com/creadores/listar/all/all/').read()
index_soup = soup(index,"html.parser")
index_link = []
for link in index_soup.find_all('a'):
    index_link.append(link.get('href'))

**3) Nos quedamos con aquellos que corresponden a links de creadores (excluimos links a otras páginas, Twitter, etc.)**

In [3]:
links_creadores = index_link[51:4676]
del links_creadores[1894]

**4) Armamos una función que toma como argumentos la lista con links y un nombre de archivo.**


In [4]:
def scrap_creadores(lista, save_file):
    creadores = []
    count = 0
    for i in lista:
        count += 1
        print count, 
        r = urllib2.urlopen(i).read()
        page_soup = soup(r,"html.parser") 
        name = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_NombreCompleto'})[0].get_text()
        cat = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_Categoria'})[0].get_text()
        fecha = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_Fechas'})[0].get_text()
        lugar = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_LugarNacimiento'})[0].get_text()
        creadores.append((i, name, cat, fecha,lugar))
        with open(save_file, 'wb') as f:
            json.dump(creadores, codecs.getwriter('utf-8')(f), ensure_ascii=False)
    return creadores

** Explicación (breve) de la función:**

4.1) Esta función recorre la lista de links 
    
    for i in lista:

4.2) descarga y parsea cada uno de los códigos html de cada página de los creadores

    r = urllib2.urlopen(i).read()
    page_soup = soup(r,"html.parser")
    
4.3) identifica la información biográfica relevante y la guarda en una variable

    name = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_NombreCompleto'})[0].get_text()
    cat = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_Categoria'})[0].get_text()
    fecha = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_Fechas'})[0].get_text()
    lugar = page_soup.findAll('span', attrs={'id':'main_fichacreador1_encabezado1_lbl_LugarNacimiento'})[0].get_text()
    
4.4) guarda todo en una lista generada previamente...
     
    creadores.append((i, name, cat, fecha,lugar))
         
4.5) y, finalmente, escribe un archivo `.txt` con la lista

    with open(save_file, 'wb') as f:
        json.dump(creadores, codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [ ]:
creador = scrap_creadores(links_creadores, "creadores.txt")

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158